In [21]:
import json
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta

In [22]:
OBP_AVG = 0.327
SLG_AVG = 0.365

FPCT_AVG = {
    '2': 0.992,
    '3': 0.993,
    '4': 0.977, 
    '5': 0.929,
    '6': 0.964,
    '7': 0.977,
    '8': 0.988,
    '9': 0.976
}

POSITION_WEIGHT = {
    '2': {"batting": 0.95, "fielding": 1.05},
    '3': {"batting": 1.21, "fielding": 0.83},
    '4': {"batting": 0.97, "fielding": 1.03},
    '5': {"batting": 0.96, "fielding": 1.04},
    '6': {"batting": 0.94, "fielding": 1.06},
    '7': {"batting": 1.02, "fielding": 0.98},
    '8': {"batting": 1.04, "fielding": 0.96},
    '9': {"batting": 1.17, "fielding": 0.85},
}

OPPONENT_WEIGHT = {
    "中信兄弟": 1.15,
    "統一7-ELEVEn獅": 1.11,
    "樂天桃猿": 0.96, 
    "富邦悍將": 0.93,
    "味全龍": 0.85
}

dataRoot = "../clean/clean_all_"
playerRoot = "../球員異動/"

teamNameMap = {
    "中信兄弟": "brothers", 
    "味全龍": "dragons",
    "富邦悍將": "guardians", 
    "統一7-ELEVEn獅": "lions",
    "樂天桃猿": "monkeys"
}

### Import Data

In [23]:
class Player:
    def __init__(self, name, data):
        self.name = name
        self.data = data

In [24]:
allData = {}
for team in teamNameMap.keys():
    with open(dataRoot + team + ".json", encoding="utf-8") as f:
        temp = []
        for line in f:
            player = json.loads(line)
            temp.append(Player(player["name"], player["data"]))
        allData[teamNameMap[team]] = temp

brothers = allData["brothers"]
dragons = allData["dragons"]
guardians = allData["guardians"]
lions = allData["lions"]
monkeys = allData["monkeys"]

In [25]:
playerList = {}

for team in teamNameMap.keys():
    tempDict = pd.read_csv(playerRoot + team + ".csv", header=0, index_col=0).to_dict()
    tempDict = {
        datetime.strptime(key, "%Y-%m-%d").date(): list(filter(lambda d: pd.isna(d) == False, list(value.values())))
        for key, value in tempDict.items()
    }
    playerList[teamNameMap[team]] = tempDict

In [26]:
def createDate(mm, dd):
    return datetime(2022, mm, dd, 0, 0, 0).date()

### Self-defined Variables

In [27]:
aTeam = "中信兄弟"
aMonth = "May"
aGame = [
    {
        "date": createDate(5, 3),
        "field": "新莊", 
        "oppo": "富邦悍將", 
        "pitcher": "陳鴻文"
    }, 
    {
        "date":createDate(5, 5), 
        "field": "澄清湖", 
        "oppo": "味全龍", 
        "pitcher": "王維中"
    }, 
    {
        "date": createDate(5, 6), 
        "field": "洲際", 
        "oppo": "統一7-ELEVEn獅", 
        "pitcher": "江辰晏"
    }, 
    {
        "date": createDate(5, 7), 
        "field": "洲際", 
        "oppo": "統一7-ELEVEn獅", 
        "pitcher": "李其峰"
    }, 
    {
        "date": createDate(5, 8), 
        "field": "洲際", 
        "oppo": "統一7-ELEVEn獅", 
        "pitcher": "邱浩鈞"
    }
]
aStar = ["許基宏", "江坤宇"]
aPotential = ["岳東華"]
aInjured = ["周思齊"]

### Calculate Parameters

In [28]:
# 比賽當天 27 人名單
def findPlayerList(team, date):
    minDelta = timedelta(days=100)
    playerDay = date
    for key in playerList[teamNameMap[aTeam]].keys():
        if date > key:
            break
        if key - date < minDelta:
            minDelta = key - date
            playerDay = key
    return playerList[teamNameMap[team]][playerDay]
        

In [29]:
# A_{ij}
def calcA(player, pos):
    if pos in player.data["fielding"]["pos"]:
        return 1
    else:
        return 0

In [30]:
# F_{ij}
def calcF(player, pos):
    F = float(player.data["fielding"]["FPCT"][str(pos)]) / FPCT_AVG[str(pos)]
    if F == 1:
        return 0.98
    if F < 0.8:
        return 0.8
    return F

In [31]:
# B_i
def calcOPS(OBP, SLG):
    return OBP / OBP_AVG + SLG / SLG_AVG - 1

def calcB(game, player):
    if game["date"].month == 4:
        aMonth = "Apr"
    if game["date"].month == 5:
        aMonth = "May"
    if game["date"].month == 6:
        aMonth = "Jun"
    aPitcher = game["pitcher"]
    aField = game["field"]

    OPSseason = float(player.data["batting"]["season"]["OPS+"])
    PAseason = player.data["batting"]["season"]["PA"]
    
    try:
        OPSmonth = calcOPS(
            float(player.data["batting"]["month"][aMonth]["OBP"]), 
            float(player.data["batting"]["month"][aMonth]["TB"]) / float(player.data["batting"]["month"][aMonth]["AB"])
        )
    except:
        OPSmonth = OPSseason
    PAmonth = player.data["batting"]["month"][aMonth]["PA"]
    
    try:
        OPSfield = calcOPS(
            float(player.data["batting"]["field"][aField]["OBP"]), 
            float(player.data["batting"]["field"][aField]["TB"]) / float(player.data["batting"]["field"][aField]["AB"])
        )
    except:
        OPSfield = OPSseason
    PAfield = player.data["batting"]["field"][aField]["PA"]
    
    try:
        OPSvsp = calcOPS(
            float(player.data["batting"]["vsP"][aPitcher]["OBP"]), 
            float(player.data["batting"]["vsP"][aPitcher]["TB"]) / float(player.data["batting"]["vsP"][aPitcher]["AB"])
        )
    except:
        OPSvsp = OPSseason
    PAvsp = player.data["batting"]["vsP"][aPitcher]["PA"]

    B = (OPSseason * PAseason + OPSmonth * PAmonth + OPSfield * PAfield + OPSvsp * PAvsp) / 400
    return B

# V_{ij}
def calcV(game, player, pos):
    A = calcA(player, pos)
    B = calcB(game, player)
    F = calcF(player, pos)
    if(str(pos) in F.keys()):
        F = F[str(pos)]
    else:
        F = 0
    return (POSITION_WEIGHT[str(pos)]["batting"] * B + POSITION_WEIGHT[str(pos)]["fielding"] * F) * A[pos - 1]

### Optimization

In [33]:
model = gp.Model("model")

# decision variables
x = []
for g in range(len(aGame)):
    todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
    singleGame = {}
    for i in range(len(todayPlayerList)):
        singlePlayer = []
        for j in range(9):
            singlePlayer.append(model.addVar(lb=0, vtype="B", name="x" + str(g) + "-" + todayPlayerList[i] + "-" + str(j + 1)))
        singleGame[todayPlayerList[i]] = singlePlayer
    x.append(singleGame)

# objective function
model.setObjective(gp.quicksum(
    gp.quicksum(
        gp.quicksum(
            OPPONENT_WEIGHT[aGame[g]["oppo"]] * calcV(aGame[g], list(filter(lambda d: d.name == findPlayerList(aTeam, aGame[g]["date"])[i], allData[teamNameMap[aTeam]]))[0], j) * x[g][i][j]
            for  j in range(2, 10)
        ) for i in range(len(findPlayerList(aTeam, aGame[g]["date"])))
    ) for g in range(len(aGame))
))

# constraints
for g in range(len(aGame)):
    todayPlayerList = findPlayerList(aTeam, aGame[g]["date"])
    for i in range(len(todayPlayerList)):
        try:
            thisPlayer = list(filter(lambda d: d.name == todayPlayerList[i], todayPlayerObject))[0]
            
            # constraint 1
            model.addConstr(calcF(player, 8) * x[g][i][8] >= calcF(player, 7) * x[g][i][7])
            # constraint 2
            model.addConstr(calcF(player, 8) * x[g][i][8] >= calcF(player, 9) * x[g][i][9])
        except:
            pass
        
        # constraint 4
        model.addConstr(sum(x[g][i][j] for j in range(2, 10)))

    for j in range(2, 10):
        # constraint 3
        model.addConstr(sum(x[g][i][j] for i in  range(len(todayPlayerList))))

model.optimize()

IndexError: list index out of range

In [34]:
for var in model.getVars():
    if var.x != 0.0:
        print(var.varName, var.x)